In [6]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [7]:
# fix random seed for rproducibility
seed = 7
numpy.random.seed(seed)

In [8]:
# Load Data
dataframe = pandas.read_csv("sonar.csv",header=None)
dataset = dataframe.values
# Split it into input[x] and output [Y] variable
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

In [9]:
# Output variable is a string value we must convert it into 0 and 1
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [31]:

# baseline model
def create_baseline():
	# create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
	# Compile model
    model.compile( optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [32]:
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 78.25% (10.73%)


In [33]:
# Evaluate baseline model with standardized dataset
numpy.random.seed(seed)
estimators=[]
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp',KerasClassifier(build_fn=create_baseline,epochs=100,batch_size=5,verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
results = cross_val_score(pipeline,X,encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 84.11% (7.25%)


In [34]:
# smaller model
def create_smaller():    
#     create model
    model = Sequential()
    model.add(Dense(30, input_dim=60, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
#     Compile Model
    model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [35]:
estimators=[]
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp',KerasClassifier(build_fn=create_smaller,epochs=100,batch_size=5,verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
results = cross_val_score(pipeline,X,encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 83.59% (7.67%)


In [36]:
# larger model
def create_larger():
	# create model
    model = Sequential()
    model.add(Dense(30, input_dim=60, activation='relu'))
    model.add(Dense(30, input_dim=30, activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
	# Compile model
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [37]:
estimators=[]
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp',KerasClassifier(build_fn=create_larger,epochs=100,batch_size=5,verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
results = cross_val_score(pipeline,X,encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 87.49% (8.13%)
